参考：https://tensorflow.google.cn/tutorials/quickstart/beginner

In [6]:
import tensorflow as tf

import numpy as np

### 加载数据集

加载并准备 MNIST 数据集。将样本数据从整数转换为浮点数：

In [7]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

### 构建机器学习模型

通过堆叠层来构建 tf.keras.Sequential 模型

In [8]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

对于每个样本，模型都会返回一个包含 logits 或 log-odds 分数的向量，每个类一个。

In [9]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.18543927, -0.25567344,  0.6349365 ,  0.26560476, -0.14295079,
        -0.9846669 ,  0.7638925 , -0.6912051 ,  0.18511018,  0.7829043 ]],
      dtype=float32)

tf.nn.softmax 函数将这些 logits 转换为每个类的概率：

In [10]:
tf.nn.softmax(predictions).numpy()

array([[0.0966991 , 0.06220847, 0.15157814, 0.10477022, 0.06963129,
        0.03000901, 0.17244136, 0.04024395, 0.09666727, 0.17575115]],
      dtype=float32)

使用 losses.SparseCategoricalCrossentropy 为训练定义损失函数，它会接受 logits 向量和 True 索引，并为每个样本返回一个标量损失。

In [11]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

此损失等于 true 类的负对数概率：如果模型确定类正确，则损失为零。

这个未经训练的模型给出的概率接近随机（每个类为 1/10），因此初始损失应该接近 -tf.math.log(1/10) ~= 2.3。

In [12]:
loss_fn(y_train[:1], predictions).numpy()

3.5062575

在开始训练之前，使用 Keras Model.compile 配置和编译模型。将 optimizer 类设置为 adam，将 loss 设置为您之前定义的 loss_fn 函数，并通过将 metrics 参数设置为 accuracy 来指定要为模型评估的指标。

In [13]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

### 训练并评估模型

使用 Model.fit 方法调整您的模型参数并最小化损失：

In [14]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 952us/step - loss: 0.2961 - accuracy: 0.9138
Epoch 2/5
1875/1875 [==============================] - 2s 988us/step - loss: 0.1443 - accuracy: 0.9569
Epoch 3/5
1875/1875 [==============================] - 2s 931us/step - loss: 0.1078 - accuracy: 0.9674
Epoch 4/5
1875/1875 [==============================] - 2s 929us/step - loss: 0.0883 - accuracy: 0.9730
Epoch 5/5
1875/1875 [==============================] - 2s 959us/step - loss: 0.0756 - accuracy: 0.9758


Model.evaluate 方法通常在 "Validation-set" 或 "Test-set" 上检查模型性能。

In [15]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0734 - accuracy: 0.9785 - 355ms/epoch - 1ms/step


[0.07344986498355865, 0.9785000085830688]

如果您想让模型返回概率，可以封装经过训练的模型，并将 softmax 附加到该模型：

In [16]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [17]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[9.66496145e-08, 9.57034274e-08, 5.36225434e-06, 8.03409421e-05,
        1.34211045e-11, 4.25583693e-07, 1.88169228e-12, 9.99901175e-01,
        3.22874058e-07, 1.22024148e-05],
       [8.64995631e-09, 4.92811785e-04, 9.99506474e-01, 5.48797914e-07,
        1.31486141e-12, 1.18242887e-07, 2.92217628e-08, 1.31661097e-15,
        5.22709609e-09, 1.42575101e-11],
       [7.47400506e-08, 9.99516129e-01, 6.99105512e-05, 3.46861248e-06,
        7.67320162e-05, 1.28332681e-06, 2.58788828e-06, 2.98048544e-04,
        3.09597817e-05, 6.82368409e-07],
       [9.99746025e-01, 2.86606827e-09, 1.86772493e-04, 4.71734694e-07,
        5.09588460e-07, 1.79990038e-05, 3.99675519e-05, 1.18269145e-06,
        2.95994198e-07, 6.73111390e-06],
       [8.37212326e-07, 2.48262761e-08, 2.03521213e-05, 9.74481935e-08,
        9.99674678e-01, 1.73651628e-07, 2.57620741e-06, 2.27040218e-05,
        4.41514736e-08, 2.78583670e-04]], dtype=float32)>